# Check Tree Traversal
Given Preorder, Inorder and Postorder traversals of some tree of size $N$. The task is to check if they are all of the same tree or not.

### Example:

Input: $N = 5$

 - preorder\[\] = $\{1, 2, 4, 5, 3\}$
 - inorder\[\] = $\{4, 2, 5, 1, 3\}$
 - postorder\[\] = $\{4, 5, 2, 3, 1\}$

Output: Yes

Explanation: All of the above three traversals are of the following tree.

           1
         /   \
        2     3
      /   \
     4     5

In [44]:
class treeNode:
    def __init__(self, val, left=None, right=None, ancestor=None):
        self.val = val
        self.left = left
        self.right = right
        self.ancestor = ancestor


class tree:
    def __init__(self, adj=None):
        """
        Only handle the case when the node values are distinct.
        """
        
        if adj is None:
            return 
        
        
        valSet, tns = set(), {}
        for val, [left, right] in adj.items():
            if val not in tns:
                tns[val] = treeNode(val)
            if left is not None and left not in tns:
                tns[left] = treeNode(left)
            if right is not None and right not in tns:
                tns[right] = treeNode(right)
            valSet.add(val)
            valSet.add(left)
            valSet.add(right)
            
            
        for val, tn in tns.items():
            if val in adj:
                left, right = adj[val]
                if left is not None:
                    tn.left = tns[left]
                    tns[left].ancestor = tn
                    valSet.remove(left)
                if right is not None:
                    tn.right = tns[right]
                    tns[right].ancestor = tn
                    valSet.remove(right)
        
        self.root = tns[valSet.pop()]


    def preOrderRec(self, root, result):
        if root is None:
            return
        result.append(root.val)
        self.preOrderRec(root.left, result)
        self.preOrderRec(root.right, result)
    
    def preOrder(self):
        result = []
        self.preOrderRec(self.root, result)
        return result
    
    
    def inOrderRec(self, root, result):
        if root is None:
            return
        
        self.inOrderRec(root.left, result)
        result.append(root.val)
        self.inOrderRec(root.right, result)
    
    def inOrder(self):
        result = []
        self.inOrderRec(self.root, result)
        return result
    
    
    def postOrderRec(self, root, result):
        if root is None:
            return
        
        self.postOrderRec(root.left, result)
        self.postOrderRec(root.right, result)
        result.append(root.val)
    
    def postOrder(self):
        result = []
        self.postOrderRec(self.root, result)
        return result

In [65]:
def recover(preOrder, inOrder):
    if len(preOrder) != len(inOrder) or set(preOrder) != set(inOrder):
        return False, None
    
    N = len(preOrder)
    
    if N == 0:
        return True, None
    
    leftMost = inOrder[0]
    
    i, pEnd, iEnd = 0, N, N
    root, ancestor = None, None
    while True:
        val = preOrder[i]
        
        # get subtree size
        tmp = iEnd - 1
        while inOrder[tmp] != val:
            tmp -= 1
        subtreeSize = iEnd - (tmp + 1)
        
        # get pre-order and in-order for the subtree
        po = preOrder[pEnd - subtreeSize: pEnd]
        io = inOrder[iEnd - subtreeSize: iEnd]
        
        successful, subtreeRoot = recover(po, io)
        if successful:
            tn = treeNode(val, right=subtreeRoot)
        else:
            return False, None
        
        if root is None: 
            root = tn
        if ancestor is not None:
            ancestor.left = tn
        
        if val == leftMost:
            break
        else:
            i += 1
            ancestor = tn
            pEnd = pEnd - subtreeSize
            iEnd = tmp
            
    return True, root

In [69]:
adj = {
    1: [2, 3],
    2: [4, 5],
    3: [6, None],
    5: [7, 8],
    8: [None, 10],
    6: [9, None]
}
T = tree(adj)
print('\n=================== ground truth ===================START')
preOrder = T.preOrder()
print(f'pre-order: {preOrder}')
inOrder = T.inOrder()
print(f'in-order: {inOrder}')
postOrder = T.postOrder()
print(f'post-order: {postOrder}')
print('=================== ground truth ===================END\n')

successful, root = recover(preOrder, inOrder)

print('\n=================== recovered ===================START')
result = []
tree().inOrderRec(root, result)
print(f'pre-order: {result}')

result = []
tree().preOrderRec(root, result)
print(f'in-order: {result}')

result = []
tree().postOrderRec(root, result)
print(f'post-order: {result}')
print('=================== recovered ===================END\n')


=================== ground truth ===================START
pre-order: [1, 2, 4, 5, 7, 8, 10, 3, 6, 9]
in-order: [4, 2, 7, 5, 8, 10, 1, 9, 6, 3]
post-order: [4, 7, 10, 8, 5, 2, 9, 6, 3, 1]
=================== ground truth ===================END


=================== recovered ===================START
pre-order: [4, 2, 7, 5, 8, 10, 1, 9, 6, 3]
in-order: [1, 2, 4, 5, 7, 8, 10, 3, 6, 9]
post-order: [4, 7, 10, 8, 5, 2, 9, 6, 3, 1]
=================== recovered ===================END



In [70]:
preOrder = [1, 5, 4, 2, 3]
inOrder = [4, 2, 5, 1, 3]
postOrder = [4, 1, 2, 3, 5]
successful, root = recover(preOrder, inOrder)
if successful:
    result = []
    tree().postOrderRec(root, result)
    print(result)

[2, 4, 5, 3, 1]
